# Numeric SAM Demo Notebook

In [15]:
import os
import json
from pathlib import Path

from pddl_plus_parser.lisp_parsers import DomainParser, ProblemParser, TrajectoryParser

from sam_learning.numeric_sam import NumericSAMLearner

os.environ["CONVEX_HULL_ERROR_PATH"] = "tests/temp_files/ch_error.txt"

In [16]:
EXAMPLES_PATH = Path("demos_data")

depot_partial_domain = DomainParser(EXAMPLES_PATH / "depot_numeric.pddl", partial_parsing=True).parse_domain()
depot_problem = ProblemParser(EXAMPLES_PATH / "pfile2.pddl", domain=depot_partial_domain).parse_problem()
depot_observation = TrajectoryParser(depot_partial_domain, depot_problem).parse_trajectory(EXAMPLES_PATH / "test_numeric_trajectory")

In [17]:
with open(EXAMPLES_PATH / "depot_fluents_map.json", "rt") as json_file:
    depot_fluents_map = json.load(json_file)


In [18]:
numeric_sam = NumericSAMLearner(depot_partial_domain, depot_fluents_map)

print(numeric_sam.partial_domain.to_pddl())

(define (domain depot)
(:requirements :typing :fluents :disjunctive-preconditions :negative-preconditions :equality :universal-preconditions)
(:types 	place locatable - object
	depot distributor - place
	truck hoist surface - locatable
	pallet crate - surface
)

(:predicates (at ?x - locatable ?y - place)
	(on ?x - crate ?y - surface)
	(in ?x - crate ?y - truck)
	(lifting ?x - hoist ?y - crate)
	(available ?x - hoist)
	(clear ?x - surface)
)

(:functions (load_limit ?t - truck)
	(current_load ?t - truck)
	(weight ?c - crate)
	(fuel-cost )
)

(:action drive
	:parameters (?x - truck ?y - place ?z - place)
	:precondition (and )
	:effect (and  
		))

(:action lift
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and )
	:effect (and  
		))

(:action drop
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and )
	:effect (and  
		))

(:action load
	:parameters (?x - hoist ?y - crate ?z - truck ?p - place)
	:precondition (and )
	:effect (an

In [19]:
learned_model, learning_metadata = numeric_sam.learn_action_model([depot_observation])
print(learning_metadata)

There are too few independent rows of data! cannot solve linear equations for action - load!
There are too few independent rows of data! cannot solve linear equations for action - unload!


{'drive': 'OK', 'lift': 'OK', 'drop': 'OK', 'load': 'OK', 'unload': 'OK', 'learning_time': '0.11106538772583008'}


In [20]:
print(learned_model.to_pddl())

(define (domain depot)
(:requirements :typing :fluents :disjunctive-preconditions :negative-preconditions :equality :universal-preconditions)
(:types 	place locatable - object
	depot distributor - place
	truck hoist surface - locatable
	pallet crate - surface
)

(:predicates (at ?x - locatable ?y - place)
	(on ?x - crate ?y - surface)
	(in ?x - crate ?y - truck)
	(lifting ?x - hoist ?y - crate)
	(available ?x - hoist)
	(clear ?x - surface)
)

(:functions (load_limit ?t - truck)
	(current_load ?t - truck)
	(weight ?c - crate)
	(fuel-cost )
)

(:action drive
	:parameters (?x - truck ?y - place ?z - place)
	:precondition (and (not (at ?x ?z))
	(at ?x ?y)(not (= ?y ?z)))
	:effect (and (at ?x ?z)
		(not (at ?x ?y))
(increase (fuel-cost ) 10.0)))

(:action lift
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and (at ?y ?p)
	(not (clear ?z))
	(on ?y ?z)
	(not (lifting ?x ?y))
	(at ?z ?p)
	(at ?x ?p)
	(clear ?y)
	(available ?x))
	:effect (and (not (at ?y ?p))
		(cl

## Option B - Learning from a trajectory file without input problem

Motivation - In some cases we might be able to create trajectories without having the problem file. For example, if we have a trajectory file from a real robot, we might not have the problem file that was used to generate it. In this case, we can use the `TrajectoryParser` to parse the trajectory file *without an input problem* and then use the `NumericSAM` to learn the action model.

In [21]:
#In this example we use trajectories generated from Minecraft simulator.

minecraft_domain = DomainParser(EXAMPLES_PATH / "basic_minecraft_domain.pddl", partial_parsing=True).parse_domain()
minecraft_observation = TrajectoryParser(minecraft_domain).parse_trajectory(EXAMPLES_PATH / "basic_minecraft.trajectory")

print(minecraft_domain)

< Domain definition: polycraft
 Requirements: [':strips', ':typing', ':negative-preconditions', ':fluents']
 Predicates: []
 Functions: ['(trees_in_map )', '(count_log_in_inventory )', '(count_planks_in_inventory )', '(count_stick_in_inventory )', '(count_sack_polyisoprene_pellets_in_inventory )', '(count_tree_tap_in_inventory )', '(count_wooden_pogo_stick_in_inventory )']
 Actions: ['get_log', 'craft_plank', 'craft_stick', 'craft_tree_tap', 'craft_wooden_pogo', 'place_tree_tap']
 Constants: [] >


In [22]:
with open(EXAMPLES_PATH / "minecraft_fluents_map.json", "rt") as json_file:
    minecraft_fluents_map = json.load(json_file)

In [23]:
minecraft_sam = NumericSAMLearner(minecraft_domain, minecraft_fluents_map)

print(minecraft_sam.partial_domain.to_pddl())

(define (domain polycraft)
(:requirements :strips :typing :negative-preconditions :fluents :disjunctive-preconditions :equality :universal-preconditions)
(:functions (trees_in_map )
	(count_log_in_inventory )
	(count_planks_in_inventory )
	(count_stick_in_inventory )
	(count_sack_polyisoprene_pellets_in_inventory )
	(count_tree_tap_in_inventory )
	(count_wooden_pogo_stick_in_inventory )
)

(:action get_log
	:parameters ()
	:precondition (and )
	:effect (and  
		))

(:action craft_plank
	:parameters ()
	:precondition (and )
	:effect (and  
		))

(:action craft_stick
	:parameters ()
	:precondition (and )
	:effect (and  
		))

(:action craft_tree_tap
	:parameters ()
	:precondition (and )
	:effect (and  
		))

(:action craft_wooden_pogo
	:parameters ()
	:precondition (and )
	:effect (and  
		))

(:action place_tree_tap
	:parameters ()
	:precondition (and )
	:effect (and  
		))

)


In [24]:
learned_model, learning_metadata = minecraft_sam.learn_action_model([minecraft_observation])
print(learning_metadata)

There are too few independent rows of data! cannot solve linear equations for action - get_log!
There are too few independent rows of data! cannot solve linear equations for action - craft_plank!
There are too few independent rows of data! cannot solve linear equations for action - craft_stick!


{'get_log': 'OK', 'craft_plank': 'OK', 'craft_stick': 'OK', 'craft_tree_tap': 'OK', 'craft_wooden_pogo': 'OK', 'place_tree_tap': 'OK', 'learning_time': '0.1657414436340332'}


In [25]:
print(learned_model.to_pddl())

(define (domain polycraft)
(:requirements :strips :typing :negative-preconditions :fluents :disjunctive-preconditions :equality :universal-preconditions)
(:functions (trees_in_map )
	(count_log_in_inventory )
	(count_planks_in_inventory )
	(count_stick_in_inventory )
	(count_sack_polyisoprene_pellets_in_inventory )
	(count_tree_tap_in_inventory )
	(count_wooden_pogo_stick_in_inventory )
)

(:action get_log
	:parameters ()
	:precondition (and (or (and (= (count_planks_in_inventory ) 0.0)
	(= (count_wooden_pogo_stick_in_inventory ) 0.0)
	(= (trees_in_map ) 2.0)
	(= (count_sack_polyisoprene_pellets_in_inventory ) 0.0)
	(= (count_tree_tap_in_inventory ) 0.0)
	(= (count_stick_in_inventory ) 0.0)
	(= (count_log_in_inventory ) 2.0))
	(and (= (count_tree_tap_in_inventory ) 0.0)
	(= (count_stick_in_inventory ) 0.0)
	(= (trees_in_map ) 4.0)
	(= (count_log_in_inventory ) 0.0)
	(= (count_wooden_pogo_stick_in_inventory ) 0.0)
	(= (count_sack_polyisoprene_pellets_in_inventory ) 0.0)
	(= (count_plank